In [18]:
!python3.11 -m pip install --upgrade pip
!pip install --upgrade lightgbm
!pip install MLB-StatsAPI
!pip install pyBMA
!pip install matplotlib scikit-learn

In [2]:
import numpy as np
import pandas as pd
import statsapi
import matplotlib.pyplot as plt
import csv
import sys

import lightgbm as lgbm
import structureboost as stb
import ml_insights as mli
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from structureboost import log_loss
pd.set_option('display.max_columns',1000)
pd.set_option('display.max_rows',1000)

In [3]:
df = pd.read_csv('df_bp9_new.csv')

/var/folders/4n/m6y9jtrx6m1_59wkdvlnxsm80000gn/T/ipykernel_2059/3503072021.py:1: DtypeWarning: Columns (13,14,15,19,85,87,159,198) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('df_bp9_new.csv')


In [4]:
df = pd.concat([df, pd.get_dummies(df['team_h'], prefix='stadium')], axis=1)

In [5]:
team_win_streak = {

}

def get_win_streak(home_team, visiting_team, home_team_win, ret_home_win_streak):

    prev_home_win_rate = 0
    prev_visitor_win_rate = 0

    if home_team in team_win_streak:
        
        prev_home_win_rate = team_win_streak[home_team]

    if visiting_team in team_win_streak:
        
        prev_visitor_win_rate = team_win_streak[visiting_team]

    if home_team not in team_win_streak:

        if home_team == 1:

            team_win_streak[home_team] = 1
            team_win_streak[visiting_team] = -1

        else:

            team_win_streak[home_team] = -1
            team_win_streak[visiting_team] = 1

    elif visiting_team not in team_win_streak:

        if home_team == 1:

            team_win_streak[home_team] = 1
            team_win_streak[visiting_team] = -1

        else:

            team_win_streak[home_team] = -1
            team_win_streak[visiting_team] = 1

    else:
        
        if home_team_win == 1:

            # The home team was on a losing streak and now won the game
            if prev_home_win_rate < 0:
                team_win_streak[home_team] = 1
            # The home team was on a winning streak and won the game
            else:
                team_win_streak[home_team] += 1

            # The visiting team was on a winning streak and now lost the game
            if prev_visitor_win_rate > 0:
                team_win_streak[visiting_team] = -1
            # The visiting team was on a losing streak and lost the game
            else:
                team_win_streak[visiting_team] += -1


        else:

            # The home team was on a winning streak and now lost the game
            if prev_home_win_rate > 0:
                team_win_streak[home_team] = -1
            # The home team was on a losing streak and lost the game
            else:
                team_win_streak[home_team] += -1

            # The visiting team was on a losing streak and now won the game
            if prev_visitor_win_rate < 0:
                team_win_streak[visiting_team] = 1
            # The visiting team was on a winning streak and won the game
            else:
                team_win_streak[visiting_team] += 1


    if ret_home_win_streak:
        return prev_home_win_rate
    else:
        return prev_visitor_win_rate

df['home_win_streak'] = df.apply(lambda row: get_win_streak(row['team_h'], row['team_v'], row['home_victory'], True), axis=1)

team_win_streak = {}

df['visitor_win_streak'] = df.apply(lambda row: get_win_streak(row['team_h'], row['team_v'], row['home_victory'], False), axis=1)

In [6]:
df['month'] = df['date'].apply(lambda x: str(x)[4:6])
df['day'] = df['date'].apply(lambda x: str(x)[6:8])
df['year'] = df['date'].apply(lambda x: str(x)[0:4])

df.month.unique()

cumulative_runs = {}
game_counts = {}

def calculate_avg(team, runs, month, season):

    if month <= '06':

        if season not in cumulative_runs:

            cumulative_runs[season] = { team: runs }
            game_counts[season] = { team: 1 }

        elif team not in cumulative_runs[season]:

            cumulative_runs[season][team] = runs
            game_counts[season][team] = 1

        else:

            cumulative_runs[season][team] += runs
            game_counts[season][team] += 1

        season = str(int(season) - 1)

        if season not in cumulative_runs:
            return None

        avg = cumulative_runs[season][team] / game_counts[season][team]

        return avg

    else:

        if season not in cumulative_runs:
            print(season, month)
            sys.exit()

        if team not in cumulative_runs[season]:
            print("ERROR: SOME HOW")
            return None

        avg = cumulative_runs[season][team] / game_counts[season][team]

        cumulative_runs[season][team] += runs
        game_counts[season][team] += 1

        return avg


df['avg_runs_v'] = df.apply(lambda row: calculate_avg(row['team_v'], row['runs_v'], row['month'], row['season']), axis=1 )
df['avg_runs_h'] = df.apply(lambda row: calculate_avg(row['team_h'], row['runs_h'], row['month'], row['season']), axis=1 )

In [7]:

cumulative_runs = {}
game_counts = {}

def calc_runs_allowed(runs_scored_on_team, team, month, season):


    if month <= '06':

        if season not in cumulative_runs:

            cumulative_runs[season] = {
                team: runs_scored_on_team
            }

            game_counts[season] = {
                team: 1
            }

        elif team not in cumulative_runs[season]:

            cumulative_runs[season][team] = runs_scored_on_team
            game_counts[season][team] = 1

        else:

            cumulative_runs[season][team] += runs_scored_on_team
            game_counts[season][team] += 1

        season = str(int(season) - 1)

        if season not in cumulative_runs:
            return None

        avg = cumulative_runs[season][team] / game_counts[season][team]

        return avg

    else:

        if season not in cumulative_runs:
            print(season, month)
            sys.exit()

        if team not in cumulative_runs[season]:
            print("ERROR: SOME HOW")
            return None

        avg = cumulative_runs[season][team] / game_counts[season][team]

        cumulative_runs[season][team] += runs_scored_on_team
        game_counts[season][team] += 1

        return avg

df['avg_runs_allowed_v'] = df.apply(lambda row: calc_runs_allowed(row['runs_h'], row['team_v'], row['month'], row['season']), axis=1 )
df['avg_runs_allowed_h'] = df.apply(lambda row: calc_runs_allowed(row['runs_v'], row['team_h'], row['month'], row['season']), axis=1 )

In [8]:
team_time_zones = {
    "TOR": "Eastern",  # Toronto Blue Jays
    "ATL": "Eastern",  # Atlanta Braves
    "BAL": "Eastern",  # Baltimore Orioles
    "DET": "Eastern",  # Detroit Tigers
    "BOS": "Eastern",  # Boston Red Sox
    "MIN": "Central",  # Minnesota Twins
    "NYA": "Eastern",  # New York Yankees
    "LAN": "Pacific",  # Los Angeles Dodgers
    "CHN": "Central",  # Chicago Cubs
    "SFN": "Pacific",  # San Francisco Giants
    "PIT": "Eastern",  # Pittsburgh Pirates
    "CLE": "Eastern",  # Cleveland Guardians
    "MON": "Eastern",  # Montreal Expos (Defunct, historically Eastern)
    "SEA": "Pacific",  # Seattle Mariners
    "KCA": "Central",  # Kansas City Royals
    "PHI": "Eastern",  # Philadelphia Phillies
    "MIL": "Central",  # Milwaukee Brewers
    "CIN": "Eastern",  # Cincinnati Reds
    "NYN": "Eastern",  # New York Mets
    "HOU": "Central",  # Houston Astros (historically Central, now in American League)
    "SLN": "Central",  # St. Louis Cardinals
    "SDN": "Pacific",  # San Diego Padres
    "CHA": "Central",  # Chicago White Sox
    "TEX": "Central",  # Texas Rangers
    "CAL": "Pacific",  # California Angels (Now Los Angeles Angels)
    "OAK": "Pacific",  # Oakland Athletics
    "COL": "Mountain", # Colorado Rockies
    "FLO": "Eastern",  # Florida Marlins (Now Miami Marlins)
    "ANA": "Pacific",  # Anaheim Angels (Now Los Angeles Angels)
    "TBA": "Eastern",  # Tampa Bay Rays
    "ARI": "Mountain", # Arizona Diamondbacks
    "WAS": "Eastern",  # Washington Nationals
    "MIA": "Eastern",  # Miami Marlins
}


from datetime import datetime
import pytz

def hour_difference(timezone1, timezone2):
    """
    Returns the hour difference between two time zones.

    Parameters:
    timezone1 (str): The name of the first time zone (e.g., 'Eastern', 'Pacific').
    timezone2 (str): The name of the second time zone (e.g., 'Central', 'Mountain').

    Returns:
    int: The hour difference between the two time zones.
    """
    # Mapping shorthand names to actual time zone names
    timezone_map = {
        "Eastern": "US/Eastern",
        "Central": "US/Central",
        "Mountain": "US/Mountain",
        "Pacific": "US/Pacific",
    }
    
    # Get the actual time zones
    tz1 = pytz.timezone(timezone_map.get(timezone1))
    tz2 = pytz.timezone(timezone_map.get(timezone2))
    
    # Get the current time in UTC
    now = datetime.utcnow()
    
    # Localize the current time to each time zone
    offset1 = tz1.utcoffset(now).total_seconds() / 3600
    offset2 = tz2.utcoffset(now).total_seconds() / 3600
    
    # Calculate and return the hour difference
    return abs(int(offset1 - offset2))


df['hour_diff'] = df.apply(lambda x: hour_difference( team_time_zones[x.team_h], team_time_zones[x.team_v]), axis=1)

In [9]:
# Step 1: Extract year from the date column (already done in your code)
df['year'] = df['date'].astype('str')
df['year'] = df['year'].apply(lambda x: int(x[:4]))

# Step 2: Calculate home win rate grouped by team and year
home_win_rate = df.groupby(['team_h', 'year']).agg({'home_victory': 'mean'}).reset_index()
home_win_rate['home_win_rate'] = home_win_rate['home_victory']
home_win_rate.drop(columns=['home_victory'], inplace=True)

# Step 3: Shift the home win rate by year to get the previous year's data
home_win_rate['prev_year_home_win_rate'] = home_win_rate.groupby('team_h')['home_win_rate'].shift(1)

# Step 4: Merge the previous year's win rate back to the original DataFrame
df = pd.merge(df, home_win_rate[['team_h', 'year', 'prev_year_home_win_rate']],
              on=['team_h', 'year'],
              how='left')

In [10]:
visitor_win_rate = df.groupby(['team_v', 'year']).agg({'home_victory': 'mean'}).reset_index()
visitor_win_rate['visitor_win_rate'] = visitor_win_rate['home_victory']
visitor_win_rate.drop(columns=['home_victory'], inplace=True)

# Step 3: Shift the home win rate by year to get the previous year's data
visitor_win_rate['prev_year_visitor_win_rate'] = visitor_win_rate.groupby('team_v')['visitor_win_rate'].shift(1)

# Step 4: Merge the previous year's win rate back to the original DataFrame
df = pd.merge(df, visitor_win_rate[['team_v', 'year', 'prev_year_visitor_win_rate']],
              on=['team_v', 'year'],
              how='left')



In [11]:
df = df[~(df.prev_year_home_win_rate.isnull()) | ~(df.prev_year_visitor_win_rate.isnull())]

## Begin Modeling

In [12]:
df = df[df.run_diff!=0]
df_train = df[(df.season>1980) & (df.season<=2016) & ~(df.OBP_162_h.isnull())]
# df_train = df[(df.season>2000) & (df.season<=2016) & ~(df.OBP_162_h.isnull())]
df_valid = df[(df.season>=2017) & (df.season<=2018)]
df_test = df[df.season>=2021]

## Let's add in some lineup features

In [13]:
target = 'home_victory'

y_train = df_train[target].to_numpy()
y_valid = df_valid[target].to_numpy()
y_test = df_test[target].to_numpy()


In [14]:

class Blending:
    def __init__(self, base_model_one, base_model_two, meta_model):
        self.base_model_one = base_model_one
        self.base_model_two = base_model_two
        self.meta_model = meta_model


    def fit(self, X_train, y_train, X_val, y_val):

        self.base_model_one.fit(X_train, y_train)
        self.base_model_two.fit(X_train, y_train)


        pred_1 = self.base_model_one.predict_proba(X_val)[:, 1]
        pred_2 = self.base_model_two.predict_proba(X_val)[:, 1]


        blending_X = np.column_stack((pred_1, pred_2))
        blending_y = y_val

        self.meta_model.fit(blending_X, blending_y)

        return self

    def predict(self, X_test):
        test_pred_1 = self.base_model_one.predict_proba(X_test)[:, 1]
        test_pred_2 = self.base_model_two.predict_proba(X_test)[:, 1]

        # Create test features for the meta-learner
        test_blending_X = np.column_stack((test_pred_1, test_pred_2))

        # Use the meta-learner to make the final predictions
        final_pred = self.meta_model.predict(test_blending_X)
        return final_pred



In [15]:
def res(pred, implied_prob):

      if abs(pred - implied_prob) >= 0.1:
            return implied_prob
      return pred

def try_features(feat_set, max_depth=2):
    target = 'home_victory'
    X_train = df_train.loc[:,feat_set]
    X_valid = df_valid.loc[:,feat_set]
    X_test = df_test.loc[:,feat_set]
    # Impute missing values
    imputer = SimpleImputer(strategy='mean')  # Or 'median', 'most_frequent', etc.
    X_train = imputer.fit_transform(X_train)
    X_valid = imputer.transform(X_valid)
    X_test = imputer.transform(X_test)



    lgbm1 = lgbm.LGBMClassifier(n_estimators=1000, learning_rate=.02, max_depth=max_depth)
    lgbm1.fit(X_train, y_train, eval_set=(X_valid, y_valid), eval_metric='logloss', 
          callbacks=[lgbm.early_stopping(stopping_rounds=50)])


    preds_lgbm_test = lgbm1.predict_proba(X_test)[:,1]
    ll_test = log_loss(y_test, preds_lgbm_test)
    result = pd.DataFrame(preds_lgbm_test, columns=['result'])

    dec_tree = DecisionTreeClassifier(max_depth=max_depth)
    log_reg = LogisticRegression(max_iter=1000)
    random_forest = RandomForestClassifier(max_depth=max_depth)

    stacking_clf = StackingClassifier(estimators=[
                                                    # ('xgb', XGBClassifier(n_estimators=1000, learning_rate=.02, max_depth=max_depth)),
                                                    # ('dec_tree', dec_tree),
                                                    ('random_forest', random_forest),
                                                    ('log_reg', log_reg)
                                                    ],
                                        final_estimator=LogisticRegression(max_iter=1000))


    blending_model = Blending(lgbm1, stacking_clf, log_reg)

    blending_model.fit(X_train, y_train, X_valid, y_valid)

    # calibrated_stacking = CalibratedClassifierCV(blending_model, method='sigmoid', cv="prefit")
    # calibrated_stacking.fit(X_valid, y_valid)

    stacking_preds = blending_model.predict(X_test)

    new_model = pd.DataFrame(stacking_preds, columns=['result'])

    implied_prob = pd.DataFrame()
    implied_prob['implied_prob_h_mid'] = df_test['implied_prob_h_mid']


    new_df = pd.DataFrame()
    new_df['our_result'] = new_model['result']
    new_df['implied_prob']  = implied_prob['implied_prob_h_mid']

#     new_df['final_result'] = new_df.apply(lambda row: res(row.our_result, row.implied_prob), axis=1)

    implied_prob['implied_prob_h_mid'] = (implied_prob['implied_prob_h_mid'] > .5).astype(int)
    result['result'] = (result['result'] > 0.5).astype(int)
    new_model['result'] = (new_model['result'] > 0.5).astype(int)

    test_df = pd.DataFrame()
    test_df['result'] = new_model['result']
    test_df['implied_prob_h_mid'] = implied_prob['implied_prob_h_mid']

#     our_accuracy_rate = round((accuracy_score(y_test, new_model['result']) * 100),2)
    our_accuracy_rate = round((accuracy_score(y_test, new_df['our_result']) * 100),2)
    yt_accuracy_rate = round((accuracy_score(y_test, result['result']) * 100), 2)

#     difference_in_accuracy = round((accuracy_score(y_test, new_model['result']) * 100) - (accuracy_score(y_test, implied_prob['implied_prob_h_mid']) * 100), 2)
    difference_in_accuracy = round((accuracy_score(y_test, new_df['our_result']) * 100) - (accuracy_score(y_test, implied_prob['implied_prob_h_mid']) * 100), 2)
    difference_in_accuracy_yt = round((accuracy_score(y_test, result['result']) * 100) - (accuracy_score(y_test, implied_prob['implied_prob_h_mid']) * 100), 2)
    vegas_accuracy_rate = round((accuracy_score(y_test, implied_prob['implied_prob_h_mid']) *100),2)

    print("YT model's accuracy score: ", yt_accuracy_rate)
    print("Vegas model's accuracy score: ", vegas_accuracy_rate)
    print("Our model's accuracy score: ", our_accuracy_rate)
    print("Difference in accuracy (YT): ", difference_in_accuracy_yt)
    print("Difference in accuracy (Our):", difference_in_accuracy)

    return yt_accuracy_rate, vegas_accuracy_rate, our_accuracy_rate, difference_in_accuracy_yt, difference_in_accuracy


In [16]:
best_features = [
        "Strt_WHIP_35_v",
        "Strt_TB_BB_perc_35_h",
        "Strt_TB_BB_perc_35_v",
        "Strt_H_BB_perc_35_h",
        "Strt_H_BB_perc_35_v",
        "Strt_SO_perc_10_h",
        "Strt_SO_perc_10_v",
        "Bpen_WHIP_75_h",
        "Bpen_WHIP_75_v",
        "Bpen_TB_BB_perc_75_h",
        "Bpen_TB_BB_perc_75_v",
        "Bpen_H_BB_perc_75_h",
        "Bpen_H_BB_perc_75_v",
        "Bpen_SO_perc_75_h",
        "Bpen_SO_perc_75_v",
        "Bpen_WHIP_35_h",
        "Bpen_WHIP_35_v",
        "Bpen_TB_BB_perc_35_h",
        "Bpen_TB_BB_perc_35_v",
        "Bpen_H_BB_perc_35_h",
        "Bpen_H_BB_perc_35_v",
        "prev_year_home_win_rate",
        "avg_runs_allowed_h",
        "avg_runs_allowed_v",
        "avg_runs_h",
        "avg_runs_v",
        "prev_year_visitor_win_rate",
        "days_between_games_h",
        # "stadium_ANA",
        # "stadium_ARI",
        # "stadium_ATL",
        # "stadium_BAL",
        # "stadium_BOS",
        # "stadium_CAL",
        # "stadium_CHA",
        # "stadium_CHN",
        # "stadium_CIN",
        # "stadium_CLE",
        # "stadium_COL",
        # "stadium_DET",
        # "stadium_FLO",
        # "stadium_HOU",
        # "stadium_KCA",
        # "stadium_LAN",
        # "stadium_MIA",
        # "stadium_MIL",
        # "stadium_MIN",
        # "stadium_MON",
        # "stadium_NYA",
        # "stadium_NYN",
        # "stadium_OAK",
        # "stadium_PHI",
        # "stadium_PIT",
        # "stadium_SDN",
        # "stadium_SEA",
        # "stadium_SFN",
        # "stadium_SLN",
        # "stadium_TBA",
        # "stadium_TEX",
        # "stadium_TOR",
        # "stadium_WAS",
        "days_between_games_v",
        "hour_diff",
        "implied_prob_h",
        "implied_prob_v",
        "implied_prob_h_mid",
        "Bpen_SO_perc_35_h",
        "Bpen_SO_perc_35_v",
        "Bpen_WHIP_10_h",
        "Bpen_WHIP_10_v",
        "Bpen_TB_BB_perc_10_h",
        "Bpen_TB_BB_perc_10_v",
        "Bpen_H_BB_perc_10_h",
        "Bpen_H_BB_perc_10_v"
    ]

### First, let's revisit our best model from our last modeling session

In [ ]:
yt_accuracy_rate, vegas_accuracy_rate, our_accuracy_rate, difference_in_accuracy_yt, difference_in_accuracy = try_features(best_features)

In [20]:
def bayesian_model_averaging(predictions, model_weights=None):
    """
    Perform Bayesian Model Averaging.
    
    predictions: numpy array of shape (num_samples, num_models), each column is a model's probability output.
    model_weights: Optional list of weights for models. If None, all models are weighted equally.
    
    Returns: Averaged probabilities.
    """
    if model_weights is None:
        model_weights = np.ones(predictions.shape[1]) / predictions.shape[1]  # Equal weight if not provided

    # Normalize weights to sum to 1
    model_weights = np.array(model_weights) / np.sum(model_weights)

    # Compute weighted average of predictions
    return np.sum(predictions * model_weights, axis=1)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

imputer = SimpleImputer(strategy='mean')  # Or 'median', 'most_frequent', etc.
df[best_features] = imputer.fit_transform(df[best_features])  # Ensure that you're only imputing relevant columns

df_test = df[df.season>=2021].loc[:, best_features + ['home_victory']]
df_train = df[(df.season>1980) & (df.season<=2016)].loc[:, best_features + ['home_victory']]
df_valid = df[(df.season>=2017) & (df.season<=2018)].loc[:, best_features + ['home_victory']]

X_train = df_train.drop(columns=['home_victory'])
X_valid = df_valid.drop(columns=['home_victory'])
X_test = df_test.drop(columns=['home_victory'])

y_train = df_train['home_victory']
y_valid = df_valid['home_victory']
y_test = df_test['home_victory']

# Train individual models
model_1 = LogisticRegression()
model_2 = RandomForestClassifier(n_estimators=100)
model_3 = SVC(probability=True)  # Enable probability output for BMA

# Fit models on training data
model_1.fit(X_train, y_train)
model_2.fit(X_train, y_train)
model_3.fit(X_train, y_train)


# Get model predictions (probabilities for class 1)
preds_1 = model_1.predict_proba(X_test)[:, 1]
preds_2 = model_2.predict_proba(X_test)[:, 1]
preds_3 = model_3.predict_proba(X_test)[:, 1]

# Stack predictions
predictions = np.column_stack((preds_1, preds_2, preds_3))

# Perform Bayesian Model Averaging
bma_preds = bayesian_model_averaging(predictions, y_test)

# Convert probabilities to binary predictions (0 or 1)
final_preds = (bma_preds >= 0.5).astype(int)

# Evaluate accuracy
accuracy = accuracy_score(y_test, final_preds)
print(f"BMA Accuracy: {accuracy:.4f}")



/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
